# Automated Battery Data Analysis

This Python code reads all the battery cycling data files (in .csv format) in a given directory and produces report files. The reports  present the "specific capacity" of each cell as a function of cycle number and illustrate the voltage profile of the cell.  

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb
import openpyxl
from openpyxl.workbook import Workbook
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, PieChart, Series, Reference
from openpyxl.styles import Font, colors, Color, Alignment, PatternFill, GradientFill, Border, Side
from openpyxl.styles import NamedStyle
import os

In [2]:
FilesPath = # Type the path to the csv files
for file in os.scandir(FilesPath):
    if os.path.splitext(file)[1] == '.csv':
        filename = file
        filename_without_extension = os.path.splitext(file)[0]
        df_data = pd.read_csv(file)
        
        # Data Cleaning ---> The first 75 lines of the csv file contains text information about the electrochemical cell. The data array starts from line 77. The 76th line contains the data column titles. 
        for i in range(0, len(df_data.columns)):
            df_data = df_data.rename(columns={df_data.columns[i]: df_data.iloc[76,i]}) 
            
        # Data Cleaning ---> Renaming the column titles:
        df_data = df_data.rename(columns={'cycle number':'Cycle_Number', '<I>/mA':'Current_mA', 'Ewe/V': 'Voltage_V', 'Q charge/mA.h': 'QCharge_mAh', 'Q discharge/mA.h': 'QDischarge_mAh', 'time/s': 'Time_s'})
       
        # Slicing the text:
        df_data_1 = df_data[[df_data.columns[0]]][0:77]
        
        # Slicing the data array:
        df_data_2 = df_data[77:]
        
        df_data_2_sliced = df_data_2[['Cycle_Number','Current_mA','x','Voltage_V','QCharge_mAh','QDischarge_mAh','Time_s']]
        
        for i in range(1, len(df_data_2_sliced.columns)-1):
            df_data_2_sliced[[df_data_2_sliced.columns[i]]]=df_data_2_sliced[[df_data_2_sliced.columns[i]]].astype('float')
        df_data_2_sliced[['Cycle_Number']] = df_data_2_sliced[['Cycle_Number']].astype('int')
        
        # Collecting the mass of active material of the cathode from the text (it is in line 30 or 31 of the text information): 
        for i in [30, 31]:
            if df_data_1.iloc[i, 0].split()[0] == 'Mass':
                Active_Mass = float(df_data_1.iloc[i, 0].split()[5])
                break
                
        df = df_data_2_sliced[['Cycle_Number', 'Current_mA', 'x', 'Voltage_V']]   
        
        # Calculating "specific capacity":
        df['ChargeCapacity_mAh.g-1']=df_data_2_sliced['QCharge_mAh']*1000/Active_Mass
        df['DischargeCapacity_mAh.g-1']=df_data_2_sliced['QDischarge_mAh']*1000/Active_Mass 
        
        # Collecting the data for a given cycle number:
        def CycleIndexedFunc(Cycle_Number):
            CycleIndexed_df = df.loc[df['Cycle_Number'] == Cycle_Number]
            return CycleIndexed_df
        def ChargeCycleIndexedFunc(Cycle_Number):
            ChargeCycleIndexed_df = CycleIndexedFunc(Cycle_Number).loc[CycleIndexedFunc(Cycle_Number)['Current_mA'] > 0]
            return ChargeCycleIndexed_df
        def DischargeCycleIndexedFunc(Cycle_Number):
            DischargeCycleIndexed_df = CycleIndexedFunc(Cycle_Number).loc[(CycleIndexedFunc(Cycle_Number)['Current_mA'] < 0)]
            return DischargeCycleIndexed_df 
        
        CycleNumberMax = max(df['Cycle_Number'])
        
        # Producing report files:
        df_report = pd.DataFrame(index=range(1,CycleNumberMax), columns=['Cycle_Number', 'ChargeCapacity_mAh.g-1', 'DischargeCapacity_mAh.g-1'])
        for Index in range(1, CycleNumberMax):
            df_report.iloc[Index-1, 0] = Index
            df_report.iloc[Index-1, 1] = int(max(ChargeCycleIndexedFunc(Index)['ChargeCapacity_mAh.g-1']))
            df_report.iloc[Index-1, 2] = int(max(DischargeCycleIndexedFunc(Index)['DischargeCapacity_mAh.g-1']))
        rcParams['figure.figsize'] = 10,10
        fig, axs = plt.subplots(2, 2, gridspec_kw={'hspace': 0.3, 'wspace': 0.2})
        axs[0, 0].plot(df_report['Cycle_Number'], df_report['ChargeCapacity_mAh.g-1'], c='b', marker = '.', markersize = 12)
        axs[0, 0].set_title('Charge',fontsize = 14, fontweight='bold',c='b')
        axs[0, 0].set_xlabel('Cycle Number', fontsize = 12)
        axs[0, 0].set_xlim(0,25)
        axs[0, 0].set_ylabel('Specific Capacity (mAh/g)', fontsize = 12)
        axs[0, 0].set_ylim(0,200)
        axs[0, 1].plot(df_report['Cycle_Number'], df_report['DischargeCapacity_mAh.g-1'], c='limegreen', marker = '.', markersize = 12)
        axs[0, 1].set_title('Discharge', fontsize = 14, fontweight='bold', c='limegreen')
        axs[0, 1].set_xlabel('Cycle Number', fontsize = 12)
        axs[0, 1].set_xlim(0,25)
        axs[0, 1].set_ylim(0,200)
        for Index in [1, 10, 20, 30, 40, 40, 50, 60, 70, 80, 90, 100]:
            axs[1, 0].plot(ChargeCycleIndexedFunc(Index)['ChargeCapacity_mAh.g-1'], ChargeCycleIndexedFunc(Index)['Voltage_V'], c='b', linewidth=3)    
        axs[1, 0].set_xlabel('Specific Capacity (mAh/g)', fontsize = 12)
        axs[1, 0].set_xlim(0,170)
        axs[1, 0].set_ylabel('Voltage (V)', fontsize = 12)
        axs[1, 0].set_ylim(1.3, 4.5)
        for Index in [1, 10, 20, 30, 40, 40, 50, 60, 70, 80, 90, 100]:
            axs[1, 1].plot(DischargeCycleIndexedFunc(Index)['DischargeCapacity_mAh.g-1'], DischargeCycleIndexedFunc(Index)['Voltage_V'], c='limegreen', linewidth=3)
        axs[1, 1].set_xlabel('Specific Capacity (mAh/g)', fontsize = 12)
        axs[1, 1].set_xlim(0,170)
        axs[1, 1].set_ylim(1.3, 4.5)
        plt.tight_layout()
        plt.savefig(filename_without_extension + '.png', dpi = 300)
        wb = Workbook()
        ws1 = wb.active
        ws1.title = 'Cycling Report'
        ws2 = wb.create_sheet('Graph')
        for r in dataframe_to_rows(df_report, index=False, header=True):
            ws1.append(r)
        from PIL import Image
        Figure = filename_without_extension + '.png'
        img = openpyxl.drawing.image.Image(Figure)
        ws2.add_image(img, 'A1')    
        img.height = img.height * 0.2
        img.width = img.width *0.2
        wb.save(filename_without_extension + '_Report.xlsx')